In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM


2023-08-22 15:15:43.326558: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-22 15:15:43.327777: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 15:15:43.354069: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 15:15:43.354846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 15:15:43.816198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()


1115394/1115394 [==============================] - 1s 1us/step


In [3]:
text = text[300000:800000]


In [4]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))


In [5]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []


In [6]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])


In [9]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool_)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1


In [12]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))


In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=4)


Epoch 1/4
651/651 [==============================] - 33s 50ms/step - loss: 2.1485
Epoch 2/4
651/651 [==============================] - 32s 49ms/step - loss: 1.7309
Epoch 3/4
651/651 [==============================] - 32s 49ms/step - loss: 1.5976
Epoch 4/4
651/651 [==============================] - 32s 49ms/step - loss: 1.5277


In [14]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [15]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated


In [16]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))


dants, i
shall bring emilia forth.

paulina:
and i am thou should he should stand the speak the changed in the brother,
the brother the stand and with the some stay,
the wife the stand the strengthing the stay the care the changed the death,
the some the surpless the shame the changed
the great the speak the change the sense
the cheer of 
oud man that did usurp his back?
forgive the chatite of york for my seave the chain?

richard:
what say my soul be strength stay the montes
with the state the speak the should be surselled the brother,
and be it a surple the brother that what i have many love,
the lady should not still be the cheek of heaven stand him of york.

leontes:
h
our behalf
armies of pestilence; and therein be caped thou dost to make her.

duke of york:
while heaven of my vain of the prove stay,
this that thou that head me soul my woman,
while be suppers and mine earl all the streepless
that be some all the sens the benith, and the sir,
she should he
should be everit of the s

In [17]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model/assets


INFO:tensorflow:Assets written to: textgenerator.model/assets
